In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time

from erddapy import ERDDAP
import numpy as np
import xarray as xr

import utils

In [ ]:
from contextlib import contextmanager
import io
from pathlib import Path
from tempfile import NamedTemporaryFile
from typing import Dict, Optional, Generator
from typing.io import BinaryIO
from urllib.parse import urlparse

from netCDF4 import Dataset
import requests
import xarray as xr

# remove shit related to auth because idk what that does

def _urlopen(url: str, **kwargs: Dict) -> BinaryIO:
    response = requests.get(url, allow_redirects=True, **kwargs)
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise requests.exceptions.HTTPError(f"{response.content.decode()}") from err
    return io.BytesIO(response.content)

def urlopen(url: str, **kwargs: Dict) -> BinaryIO:
    """Thin wrapper around requests get content.
    See requests.get docs for the `params` and `kwargs` options.
    """
    # Ignoring type checks here b/c mypy does not support decorated functions.
    data = _urlopen(url=url, **kwargs)  # type: ignore
    data.seek(0)
    return data

def _tempnc(data: BinaryIO) -> Generator[str, None, None]:
    """Creates a temporary netcdf file."""
    tmp = None
    try:
        tmp = NamedTemporaryFile(suffix=".nc", prefix="erddapy_")
        tmp.write(data.read())
        tmp.flush()
        yield tmp.name
    finally:
        if tmp is not None:
            tmp.close()

def _nc_dataset(url, auth, **requests_kwargs: Dict):
    """Returns a netCDF4-python Dataset from memory and fallbacks to disk if that fails."""

    data = urlopen(url=url)
    try:
        return Dataset(Path(urlparse(url).path).name, memory=data.read())
    except OSError:
        # if libnetcdf is not compiled with in-memory support fallback to a local tmp file
        with _tempnc(data) as _nc:
            return Dataset(_nc)

def _quote_string_constraints(kwargs: Dict) -> Dict:
    """
    For constraints of String variables,
    the right-hand-side value must be surrounded by double quotes.
    """
    return {k: f'"{v}"' if isinstance(v, str) else v for k, v in kwargs.items()}

def _format_constraints_url(kwargs: Dict) -> str:
    """
    Join the constraint variables with separator '&' to add to the download link.
    """
    return "".join([f"&{k}{v}" for k, v in kwargs.items()])

def _distinct(url: str, **kwargs: Dict) -> str:
    """
    Sort all of the rows in the results table
    (starting with the first requested variable,
    then using the second requested variable if the first variable has a tie, ...),
    then remove all non-unique rows of data.
    For example, a query for the variables ["stationType", "stationID"] with `distinct=True`
    will return a sorted list of "stationIDs" associated with each "stationType".
    See https://coastwatch.pfeg.noaa.gov/erddap/tabledap/documentation.html#distinct
    """
    distinct = kwargs.pop("distinct", False)
    if distinct is True:
        return f"{url}&distinct()"
    return url

def get_download_url_griddap(
    server,
    dataset_id,
    variables=None,
    constraints=None,
    relative_constraints=None,
    **kwargs
) -> str:
    """The download URL for the `server` endpoint.
    Args:
        server: server url
        dataset_id: a dataset unique id.
        variables (list/tuple): a list of the variables to download.
        constraints (dict): download constraints, default None (opendap-like url)
        example: constraints = {'latitude<=': 41.0,
                                'latitude>=': 38.0,
                                'longitude<=': -69.0,
                                'longitude>=': -72.0,
                                'time<=': '2017-02-10T00:00:00+00:00',
                                'time>=': '2016-07-10T00:00:00+00:00',}
        relative_constraints (dict): advanced download constraints , default None
        example: relative_constraints = {'time>': 'now-7days',
                                     'latitude<':'min(longitude)+180'
                                     'depth>':'max(depth)-23'
                                        }
    Returns:
        url (str): the download URL for the `response` chosen.
    """
    if not dataset_id:
        raise ValueError(f"Please specify a valid `dataset_id`, got {dataset_id}")

    url = f"{server}/griddap/{dataset_id}.nc?"

    if variables:
        url += ",".join(variables)

    if constraints:
        _constraints = copy.copy(constraints)
        for k, v in _constraints.items():
            if k.startswith("time"):
                _constraints.update({k: parse_dates(v)})
        _constraints = _quote_string_constraints(_constraints)
        _constraints_url = _format_constraints_url(_constraints)

        url += f"{_constraints_url}"

    if relative_constraints:
        _relative_constraints_url = _format_constraints_url(relative_constraints)
        url += f"{_relative_constraints_url}"

    url = _distinct(url, **kwargs)
    return url

def to_xr_nc4(
    server,
    dataset_id,
    variables=None,
    constraints=None,
    relative_constraints=None,
    **kwargs
) -> xr.backends.NetCDF4DataStore:
    """Load the data request into a xarray.backends.NetCDF4DataStore."""
    url = get_download_url_griddap(
        server, dataset_id, variables, constraints, relative_constraints, **kwargs
    )
    nc = _nc_dataset(url)
    return xr.backends.NetCDF4DataStore(nc)

In [ ]:
erddap = ERDDAP(
    server="https://erddap.sccoos.org/erddap",
    protocol="griddap",
    response="nc",
)
erddap.dataset_id = "roms_ncst"
erddap.variables = [
    "latitude",
    "time",
]
erddap.constraints = {
    "time>=": "2016-09-03T00:00:00Z",
    "time<=": "2016-09-04T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
}
erddap.get_download_url()

In [ ]:
ds = erddap.to_xarray()

In [ ]:
ds

In [ ]:
erddap_path = utils.CURRENT_NETCDF_DIR / "roms_ncst_ae30_8a12_e3ca.nc"
dataset = xr.open_dataset(erddap_path)

In [ ]:
dataset

In [ ]:
dataset["u"]